In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
import seaborn as sns
import pickle
from tqdm import tqdm

import sys
sys.path.insert(0, '../')
from utils.evaluation import read_base_results, read_drp_results, read_eval_results

sns.set()
sns.set_style('whitegrid')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

%load_ext autoreload
%autoreload 2

## Adult Income Dataset

In [ ]:
datasetname='adult'

#### Ensemble 

In [ ]:
file_path = '../results/'
base_file = datasetname+'-base-100-1-1000'
bdrp_ensemble_file = datasetname+'-bernoulli-dropout-10000-0.2-1,2,5,10,20,50,100-100-ensemble.npz'
gdrp_ensemble_file = datasetname+'-gaussian-dropout-10000-0.6-1,2,5,10,20,50,100-100-ensemble.npz'

base_result = read_base_results(file_path+base_file)
bdrp_ensemble_result = np.load(file_path+bdrp_ensemble_file)
gdrp_ensemble_result = np.load(file_path+gdrp_ensemble_file)

In [ ]:
ensemble_size = bdrp_ensemble_result['ensemble_size']

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(7, 4), gridspec_kw={'height_ratios': [3.5,1]})
colors = ['darkgoldenrod', 'darkorange', 'tomato', 'navy', 'royalblue', 'lightsteelblue']

q = [0.5, 0.99, 0.5, 0.9]
ax[0, 0].plot(ensemble_size, np.quantile(bdrp_ensemble_result['vpr'], q[0], axis=1), label='VPR', color=colors[0])
ax[0, 0].plot(ensemble_size, np.quantile(bdrp_ensemble_result['score_var'], q[1], axis=1), label='Score Var.', color=colors[1])
ax[0, 0].plot(ensemble_size, 2**np.quantile(bdrp_ensemble_result['rc'], q[2], axis=1), label='RC', color=colors[2])
ax[0, 0].plot(ensemble_size, np.quantile(bdrp_ensemble_result['disa_hat'], q[3], axis=1), label='Disagreement', color=colors[3])
ax[0, 0].plot(ensemble_size, bdrp_ensemble_result['amb'], label='Ambiguity', color=colors[4])
ax[0, 0].plot(ensemble_size, bdrp_ensemble_result['disc'], label='Discrepancy', color=colors[5])
ax[0, 0].set_xticks(ensemble_size);
ax[0, 0].set_title(r'Bernoulli Dropout with $p=0.2$');
ax[0, 0].set_xscale('log')
# ax[0, 0].legend(loc='upper right');
ax[0, 0].set_ylabel('Predictive Multiplicity' '\n' 'Metrics')

ax[1, 0].errorbar(x=ensemble_size, y=bdrp_ensemble_result['ensemble_acc'].mean(axis=1), yerr=bdrp_ensemble_result['ensemble_acc'].std(axis=1),
                  linestyle='-', markersize=3, fmt='s', capsize=1, capthick=1, c='maroon')
ax[1, 0].set_xscale('log')
ax[1, 0].set_xlabel('Ensemble Size (Log)')
ax[1, 0].set_ylabel('Ensemble' '\n' 'Acc.')

ax[0, 1].plot(ensemble_size, np.quantile(gdrp_ensemble_result['vpr'], q[0], axis=1), label='VPR', color=colors[0])
ax[0, 1].plot(ensemble_size, np.quantile(gdrp_ensemble_result['score_var'], q[1], axis=1), label='Score Var.', color=colors[1])
ax[0, 1].plot(ensemble_size, 2**np.quantile(gdrp_ensemble_result['rc'], q[2], axis=1), label='RC', color=colors[2])
ax[0, 1].plot(ensemble_size, np.quantile(gdrp_ensemble_result['disa_hat'], q[3], axis=1), label='Disa.', color=colors[3])
ax[0, 1].plot(ensemble_size, gdrp_ensemble_result['amb'], label='Amb.', color=colors[4])
ax[0, 1].plot(ensemble_size, gdrp_ensemble_result['disc'], label='Disc.', color=colors[5])
ax[0, 1].set_xticks(ensemble_size);
ax[0, 1].set_title(r'Gaussian Dropout with $\alpha=0.6$');
ax[0, 1].set_xscale('log')
ax[0, 1].legend(loc='center right', ncol=2, fontsize=9);

ax[1, 1].errorbar(x=ensemble_size, y=gdrp_ensemble_result['ensemble_acc'].mean(axis=1), yerr=bdrp_ensemble_result['ensemble_acc'].std(axis=1),
                  linestyle='-', markersize=3, fmt='s', capsize=1, capthick=1, c='maroon')
ax[1, 1].set_xscale('log')
ax[1, 1].set_xlabel('Ensemble Size (Log)')

plt.tight_layout()
# plt.savefig('adult-ensemble.png', format='png', dpi=300)

#### Model Selection

##### Bernoulli

In [ ]:
file_path = '../results/'
bdrp_ms_file = datasetname+ '-10-100-bernoulli-model-selection-eval.npz' 
bdrp_ms_result = np.load(file_path+bdrp_ms_file)

In [ ]:
bdrp_ms_result['drp_list']

In [ ]:
nmodel = bdrp_ms_result['vpr'].shape[0]
drp_idx = 1
print(bdrp_ms_result['drp_list'][drp_idx])

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(7, 4))

q = [0.5, 0.99, 0.5, 0.9]
xlabels = ['VPR', 'Score Var.', 'RC', 'Disa.', 'Amb.', 'Disc.']
model_idx = [0, 4, 5, 6, 7]

colors = cm.plasma(np.linspace(0, 1, len(model_idx)))

for i, idx in enumerate(model_idx):
    ax[0].scatter(1, np.quantile(bdrp_ms_result['vpr'][idx, drp_idx, :], q[0], axis=0), color=colors[i], label='Model {:2d}'.format(i+1))
    ax[1].scatter(1, np.quantile(bdrp_ms_result['score_var'][idx, drp_idx, :], q[1], axis=0), color=colors[i], label='Model {:2d}'.format(i+1))
    ax[2].scatter(1, 2**np.quantile(bdrp_ms_result['rc'][idx, drp_idx, :], q[2], axis=0), color=colors[i], label='Model {:2d}'.format(i+1))
    ax[3].scatter(1, np.quantile(bdrp_ms_result['disa_hat'][idx, drp_idx, :], q[3], axis=0), color=colors[i], label='Model {:2d}'.format(i+1))
    ax[4].scatter(1, bdrp_ms_result['amb'][idx, drp_idx], color=colors[i], label='Model {:2d}'.format(i+1))
    ax[5].scatter(1, bdrp_ms_result['disc'][idx, drp_idx], color=colors[i], label='Model {:2d}'.format(i+1))

ax[5].legend(loc='center right', bbox_to_anchor=(1.0, 0.5))

for i in range(6): ax[i].grid(False)
for i in range(6): ax[i].set_xticks([])
for i in range(6): ax[i].set_xlabel(xlabels[i])

plt.tight_layout()
# plt.savefig('adult-ms-b.png', format='png', dpi=300, bbox_inches='tight')

##### Gaussian

In [ ]:
file_path = '../results/'
gdrp_ms_file = datasetname+ '-10-100-gaussian-model-selection-eval.npz' 
gdrp_ms_result = np.load(file_path+gdrp_ms_file)

In [ ]:
gdrp_ms_result['drp_list']

In [ ]:
nmodel = gdrp_ms_result['vpr'].shape[0]
drp_idx = 20
print(gdrp_ms_result['drp_list'][drp_idx])

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(7, 4))

q = [0.5, 0.99, 0.9, 0.9]
xlabels = ['VPR', 'Score Var.', 'RC', 'Disa.', 'Amb.', 'Disc.']
model_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

colors = cm.plasma(np.linspace(0, 1, len(model_idx)))

for i, idx in enumerate(model_idx):
    ax[0].scatter(1, np.quantile(gdrp_ms_result['vpr'][idx, drp_idx, :], q[0], axis=0), color=colors[i])
    ax[1].scatter(1, np.quantile(gdrp_ms_result['score_var'][idx, drp_idx, :], q[1], axis=0), color=colors[i])
    ax[2].scatter(1, 2**np.quantile(gdrp_ms_result['rc'][idx, drp_idx, :], q[2], axis=0), color=colors[i])
    ax[3].scatter(1, np.quantile(gdrp_ms_result['disa_hat'][idx, drp_idx, :], q[3], axis=0), color=colors[i])
    ax[4].scatter(1, gdrp_ms_result['amb'][idx, drp_idx], color=colors[i])
    ax[5].scatter(1, gdrp_ms_result['disc'][idx, drp_idx], color=colors[i], label='Model {:2d}'.format(i+1))



for i in range(6): ax[i].grid(False)
for i in range(6): ax[i].set_xticks([])
for i in range(6): ax[i].set_xlabel(xlabels[i])


ax[5].legend(loc='center right', bbox_to_anchor=(1.0, 0.5))

plt.tight_layout()
# plt.savefig('adult-ms-g.png', format='png', dpi=300, bbox_inches='tight')